# Installing requirements

The USPTO datasets are in MoleculeNet and the infrastructure for tokenizing the reactions is provided by the Reaction featurizer wrapper in DeepChem. The featurizer uses the RobertaTokenizerFast in its backend, which requires the HuggingFace library to be installed.
The model and the pre-trained weights are also loaded from HuggingFace. 

In [ ]:
!pip install --pre deepchem
!pip install --pre deepchem[torch]
!pip install transformers
!pip install rdkit-pypi

     |████████████████████████████████| 609 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 5.1 MB/s 
     |████████████████████████████████| 4.5 MB 45.8 MB/s 
  Created wheel for dgllife: filename=dgllife-0.2.8-py3-none-any.whl size=213162 sha256=6ff2aa6fb34c6dff8145a68f927ca8e7b85bcf7cb7f5be72d1804f9bf02ff296
  Stored in directory: /root/.cache/pip/wheels/b6/81/6f/6bbc4b7a80b06d92829b5362a78277173c9e7b46d294d4ca45
Successfully built dgllife
     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 636 kB 46.9 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.7 MB 1.3 MB/s 


In [ ]:
import deepchem as dc
from deepchem.molnet.load_function import uspto_datasets
from transformers import RobertaTokenizerFast

# Loading and tokenizing the datasets

The USPTO datasets can be loaded in using the ```load_uspto``` method inside MoleculeNet. The loader can load in the entire USPTO dataset consisting of over a million reactions or it can also load in different subsets such as the STEREO, MIT and 50K. The loader also provides support for separating the reagents from the reactants for training.  

In [ ]:
tasks, datasets, transformers = uspto_datasets.load_uspto(subset="MIT", splitter= "random", sep_reagent=True)

Downloading:   0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
train, test, valid = datasets

<DiskDataset X.shape: (383228, 2, 2, 1), y.shape: (), w.shape: (), task_names: []>

In [ ]:
print(f"The train dataset : {train}")
print(f"Some reactions from the training set:\n {train.X[2]}")

The train dataset : <DiskDataset X.shape: (383228, 2, 2, 1), y.shape: (), w.shape: (), task_names: []>
Some reactions from the training set:
 [[[list([0, 304, 263, 51, 13, 71, 21, 261, 12, 50, 13, 308, 21, 39, 18, 51, 33, 39, 21, 267, 263, 51, 13, 50, 21, 288, 34, 291, 12, 39, 13, 39, 33, 51, 18, 51, 2])]
  [list([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]]

 [[list([0, 304, 263, 51, 13, 71, 21, 261, 12, 50, 13, 71, 12, 288, 13, 272, 21, 39, 2])]
  [list([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]]]


In [ ]:
from transformers import EncoderDecoderModel

# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("seyonec/ChemBERTa-zinc-base-v1", "seyonec/ChemBERTa-zinc-base-v1", tie_encoder_decoder=True)

Downloading:   0%|          | 0.00/501 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/171M [00:00<?, ?B/s]

Some weights of the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['roberta.encoder.layer.2.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.4.crossattention.self.

In [ ]:
#model training

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=16,  # set to 500 for full training
    eval_steps=4,  # set to 8000 for full training
    warmup_steps=1,  # set to 2000 for full training
    max_steps=16, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, 
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

NameError: ignored